# building of the shocks with normal law for the relative changes / returns
it means the returns are stationary but not deposits process which is a pure random walk log normal

In [ ]:
from scipy import stats
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# define middle of the list of banks
deposits = np.random.rand(int(1e5))*500
plt.hist(deposits)
N_max = len(deposits) - len(deposits) % 2  # can not apply a shock on
# one bank if odd nb
N_half = int(len(deposits) / 2)

In [ ]:
rho_1 = stats.truncnorm(
            (norm_lower - mu) / vol, (norm_upper - mu) / vol, loc=mu, scale=vol
        ).rvs(N_half)

In [ ]:
vol = 0.3
mu=0
norm_lower=-1
norm_upper=0

rho_1 = stats.truncnorm(
        (norm_lower - mu) / vol,
        (norm_upper - mu) / vol,
        loc=mu,
        scale=vol).rvs(N_half)
plt.figure(figsize=(15,5))
bin = plt.hist(rho_1, bins =200)
print(np.mean(rho_1),np.var(rho_1), np.max(rho_1), np.min(rho_1))

In [ ]:
# apply a positive relative shock on the second half of the banks
rho_2 = -rho_1 * deposits[0:N_half] / deposits[N_half:N_max]
plt.figure(figsize=(15,5))
bin = plt.hist(rho_2, bins =200)
print(np.mean(rho_2),np.var(rho_2), np.max(rho_2), np.min(rho_2))

In [ ]:
# concatenate the relative shocks
if len(deposits) > N_max:
    rho = np.concatenate([rho_1, rho_2, [0]])
elif len(deposits) == N_max:
    rho = np.concatenate([rho_1, rho_2])
else:
    assert False, ""

print(np.mean(rho),np.var(rho), np.max(rho), np.min(rho))
plt.figure(figsize=(15,5))
bin = plt.hist(rho, bins =200)


In [ ]:
# compute the absolute shock from the deposit amount
shocks = deposits * rho
plt.figure(figsize=(15,5))
bin = plt.hist(shocks, bins =200)

we do obtain something close to a log normal for the distribution of the deposits 

In [ ]:
# compute the new deposits
new_deposits = deposits + shocks
plt.figure(figsize=(15,5))
bin = plt.hist(new_deposits, bins =200)

In [ ]:
new_deposits

# partial stationarisation with mean reverting
the processus for the deposits is non stationary in any case but lux proposes to add a mean reverting component to make it more realistic
not implemented yet

In [ ]:
walk = np.zeros(int(1e2))
walk[0] = 1
for i in range(1,int(1e2)):
    walk[i] = 0.9*(100-walk[i-1]) + walk[i-1]*np.random.normal(scale=0.1)

plt.figure(figsize=(15,5))
plt.plot(walk,'o-')
print(np.mean(np.log(walk)),np.var(np.log(walk)), np.max(np.log(walk)), np.min(np.log(walk)))


In [ ]:
bin  = plt.hist(walk, bins=50)

# Dummy tests

In [61]:
len(np.arange(0.01, 1, 0.02))

50

In [58]:
np.logspace(-3, 2, num=25)

array([1.00000000e-03, 1.61559810e-03, 2.61015722e-03, 4.21696503e-03,
       6.81292069e-03, 1.10069417e-02, 1.77827941e-02, 2.87298483e-02,
       4.64158883e-02, 7.49894209e-02, 1.21152766e-01, 1.95734178e-01,
       3.16227766e-01, 5.10896977e-01, 8.25404185e-01, 1.33352143e+00,
       2.15443469e+00, 3.48070059e+00, 5.62341325e+00, 9.08517576e+00,
       1.46779927e+01, 2.37137371e+01, 3.83118685e+01, 6.18965819e+01,
       1.00000000e+02])

# Analysis beta init issue

In [16]:
import shocks as sh
from network import ClassNetwork
import numpy as np

In [44]:
network = ClassNetwork(
    n_banks=2,
    alpha=0.01,
    beta_init=0.01,
    beta_reg=0.1,
    beta_star=0.1,
    gamma=0.03,
    collateral_value=1.0,
    initialization_method="constant",
    alpha_pareto=1.3,
    shocks_method="bilateral",
    shocks_law="normal",
    shocks_vol=0.01,
    result_location="./results/test/",
    min_repo_size=1e-10,
    )

network.banks

c:\Users\VictorLeCoz\OneDrive - Quant Decisions S.L\Documents\PhD\secured-inter-bank-network\network.py:502: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(weighted_repo_maturity) / total_repo_amount
c:\Users\VictorLeCoz\OneDrive - Quant Decisions S.L\Documents\PhD\secured-inter-bank-network\network.py:615: RuntimeWarning: invalid value encountered in double_scalars
  self.single_trajectory["Maturity of repos"][-1] = np.sum(


[<bank.ClassBank at 0x278ac4d13a0>, <bank.ClassBank at 0x278ac4d13d0>]

# Line by line analysis

In [45]:
network.reset_network()
print(network.banks[0])

Bank 0
        ----------------------------------------
        Total Assets: 103.09
        ----Cash: 1.0
        ----Securities Usable: 0.0
        ----Securities Encumbered: 0.0
        ----Reverse Repos: 0.0
        ----Loans: 102.09
        ----------------------------------------
        Total Liabilities: 103.09
        ----Own Funds: 3.09
        ----Deposits: 100.0
        ----Repos: 0.0
        ----MROs: 0.0
        ----------------------------------------
        Off-balance sheet Items:
        ----Collateral received: 0.0
        ----Collateral Reuse: 0.0
        ----------------------------------------
        Regulatory:
        ----Liquidity Coverage Ratio: 10.0%
        ----Cash / Deposits (for minimal reserves): 1.0%
        ----Leverage / Solvency ratio: 3.0%
        


## step_network()

In [46]:
# Generation of the shocks
shocks = sh.generate_bilateral_shocks(
        network.network_deposits, law=network.shocks_law, vol=network.shocks_vol)

# For loops over the instances of ClassBank in the ClassNetwork.
ix = np.arange(network.n_banks)  # Defines an index of the banks
for i in ix:
    network.banks[i].set_shock(shocks[i])
    print(shocks[i])




1.403235798885991
-1.403235798885991


In [48]:
print(network.banks[1].shock)
print(network.banks[1])


-1.403235798885991
Bank 1
        ----------------------------------------
        Total Assets: 101.69
        ----Cash: -0.4
        ----Securities Usable: 0.0
        ----Securities Encumbered: 0.0
        ----Reverse Repos: 0.0
        ----Loans: 102.09
        ----------------------------------------
        Total Liabilities: 101.69
        ----Own Funds: 3.09
        ----Deposits: 98.6
        ----Repos: 0.0
        ----MROs: 0.0
        ----------------------------------------
        Off-balance sheet Items:
        ----Collateral received: 0.0
        ----Collateral Reuse: 0.0
        ----------------------------------------
        Regulatory:
        ----Liquidity Coverage Ratio: -4.09%
        ----Cash / Deposits (for minimal reserves): -0.41%
        ----Leverage / Solvency ratio: 3.04%
        


In [49]:
for i in ix:
    network.banks[i].step_lcr_mgt()

print(network.banks[1].shock)
print(network.banks[1])

-1.403235798885991
Bank 1
        ----------------------------------------
        Total Assets: 111.95
        ----Cash: 9.86
        ----Securities Usable: 0.0
        ----Securities Encumbered: 0.0
        ----Reverse Repos: 0.0
        ----Loans: 102.09
        ----------------------------------------
        Total Liabilities: 111.95
        ----Own Funds: 3.09
        ----Deposits: 98.6
        ----Repos: 0.0
        ----MROs: 10.26
        ----------------------------------------
        Off-balance sheet Items:
        ----Collateral received: 0.0
        ----Collateral Reuse: 0.0
        ----------------------------------------
        Regulatory:
        ----Liquidity Coverage Ratio: 100.0%
        ----Cash / Deposits (for minimal reserves): 10.0%
        ----Leverage / Solvency ratio: 2.76%
        


In [50]:

print(shocks[i])


ix = np.random.permutation(ix)  # Permutation of the
# banks' indexes to decide in which order banks can close their repos.
for i in ix:
    network.banks[i].step_end_repos()  # Run the step end repos for the bank network

print(network.banks[0])

-1.403235798885991
Bank 0
        ----------------------------------------
        Total Assets: 112.23
        ----Cash: 10.14
        ----Securities Usable: 0.0
        ----Securities Encumbered: 0.0
        ----Reverse Repos: 0.0
        ----Loans: 102.09
        ----------------------------------------
        Total Liabilities: 112.23
        ----Own Funds: 3.09
        ----Deposits: 101.4
        ----Repos: 0.0
        ----MROs: 7.74
        ----------------------------------------
        Off-balance sheet Items:
        ----Collateral received: 0.0
        ----Collateral Reuse: 0.0
        ----------------------------------------
        Regulatory:
        ----Liquidity Coverage Ratio: 100.0%
        ----Cash / Deposits (for minimal reserves): 10.0%
        ----Leverage / Solvency ratio: 2.76%
        


In [18]:


ix = np.random.permutation(ix)  # New permutation of the
# banks' indexes to decide in which order banks can enter into repos
for i in ix:
    network.banks[i].step_enter_repos()
for i in ix:
    network.banks[i].assert_minimum_reserves()
    network.banks[i].assert_alm()
    network.banks[i].assert_lcr()
    # network.banks[i].assert_leverage()
    network.banks[i].steps += 1

Bank 0
        ----------------------------------------
        Total Assets: 102.39
        ----Cash: -39.07
        ----Securities Usable: 49.0
        ----Securities Encumbered: 0.0
        ----Reverse Repos: 0.0
        ----Loans: 92.46
        ----------------------------------------
        Total Liabilities: 102.39
        ----Own Funds: 3.09
        ----Deposits: 99.3
        ----Repos: 0.0
        ----MROs: 0.0
        ----------------------------------------
        Off-balance sheet Items:
        ----Collateral received: 0.0
        ----Collateral Reuse: 0.0
        ----------------------------------------
        Regulatory:
        ----Liquidity Coverage Ratio: 100.0%
        ----Cash / Deposits (for minimal reserves): -39.35%
        ----Leverage / Solvency ratio: 3.02%
        


<>:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\VictorLeCoz\AppData\Local\Temp\ipykernel_12956\1964417710.py:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


AssertionError: repo request unsatified for bank 1, for the amount 39.93675462813507